In [5]:
import pandas as pd
import math
from collections import defaultdict

# Load the data files for each exchange (example paths, replace with your actual files)
# Each file should be in a consistent format and include columns: timestamp, close, exchange
#binance_df = pd.read_csv('path_to_binance_file.csv')
#kucoin_df = pd.read_csv('path_to_kucoin_file.csv')
okx_df = pd.read_csv(r'C:\Users\avivh\Desktop\פרוייקט גמר\Data\Pepe\התמקדות של דקה\OKX_PEPE_1.5.23_1min.csv')
huobi_df = pd.read_csv(r'C:\Users\avivh\Desktop\פרוייקט גמר\Data\Pepe\התמקדות של דקה\Houbi_PEPE_1.5.23_1min.csv')
#coinbase_df = pd.read_csv('path_to_coinbase_file.csv')

# List of exchanges and dataframes
exchange_dfs = {
 #   "Binance": binance_df,
  #  "KuCoin": kucoin_df,
    "OKX": okx_df,
    "Huobi": huobi_df,
   # "Coinbase": coinbase_df
}

# Ensure each dataframe has a common 'exchange' column
for exchange, df in exchange_dfs.items():
    df['exchange'] = exchange

# Concatenate all dataframes into one
df = pd.concat(exchange_dfs.values())

# Arbitrage Detection Function
def find_arbitrage(df, coin_name):
    # Filter data for the specified coin
    df['timestamp'] = pd.to_datetime(df['timestamp'])  # Ensure timestamp is in datetime format
  #  df = df[df['coin'] == coin_name]  # Filter for the specific coin if there are multiple

    arbitrage_opportunities = []
    
    # Iterate over each unique timestamp
    for timestamp in df['timestamp'].unique():
        # Filter data for the current timestamp
        snapshot = df[df['timestamp'] == timestamp]
        
        # Extract exchanges and their respective prices at this timestamp
        exchanges = snapshot['exchange'].tolist()
        prices = snapshot['open'].tolist()
        
        # Initialize the distance dictionary for Bellman-Ford
        distances = {exch: float('inf') for exch in exchanges}
        distances[exchanges[0]] = 0  # Arbitrary starting exchange (set distance to 0)
        
        # Create graph edges based on negative log price ratios
        edges = []
        for i in range(len(exchanges)):
            for j in range(len(exchanges)):
                if i != j:  # Avoid self-loops
                    u = exchanges[i]
                    v = exchanges[j]
                    weight = -math.log(prices[j] / prices[i])
                    edges.append((u, v, weight))
        
        # Bellman-Ford algorithm
        for _ in range(len(exchanges) - 1):  # Relax all edges N-1 times
            for u, v, weight in edges:
                if distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight

        # Check for negative-weight cycles indicating arbitrage opportunities
        for u, v, weight in edges:
            if distances[u] + weight < distances[v]:  # Negative cycle found
                arbitrage_opportunities.append((timestamp, u, v))
                print(f"Arbitrage opportunity detected at {timestamp} between {u} and {v}")
    
    return arbitrage_opportunities

# Run the function for a specific coin (replace 'COIN_NAME' with your coin of interest)
arbitrage_opportunities = find_arbitrage(df, coin_name='COIN_NAME')

# Output all arbitrage opportunities found
if not arbitrage_opportunities:
    print("No arbitrage opportunities detected for the given data.")
else:
    print("Arbitrage opportunities found:", arbitrage_opportunities)


Arbitrage opportunity detected at 2023-01-05 09:00:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:01:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:02:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:03:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:04:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:07:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:10:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:11:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:16:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:18:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:20:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:21:00 between OKX and Huobi
Arbitrage opportunity detected at 2023-01-05 09:23:00 between OKX and Huobi
Arbitrage op